# Cálculo huella carbono

In [63]:
import pandas as pd
from unidecode import unidecode

## Peso y volumen de los libros

El peso de las 500 unidades de libros incluye el peso de un europalet de 25 kg.

In [64]:
libros = pd.read_csv('../data/raw/PesoVolumenLibros.csv', encoding='latin1')
display(libros)

,libro_titulo,libro_isbn,ancho_cm,alto_cm,grosor_cm,paginas,vol_cm3_1libro,vol_m3_500libros,peso_g_1libro,peso_sineuropalet_t_500libros,peso_t_500libros
0,Niebla de sueño,978-84-1353-114-4,13,19,1.2,162,296.4,0.14820,250,0.125,0.150
1,La isla de los canarios. Volumen 6,978-84-1353-115-1,21,23,0.8,92,386.4,0.19320,400,0.200,0.225
2,El mundo del libro en Canarias,84-8103-396-0,15,21,3.1,532,976.5,0.48825,780,0.390,0.415
3,Historia del Cabildo insular de Gran Canaria,84-8103-067-8,17,24,3.9,748,1591.2,0.79560,1240,0.620,0.645
4,Flora de Gran Canaria 1,978-84-8103-741,27,34,3.0,238,2754.0,1.37700,2280,1.140,1.165


## Etapa 1: identificación de los diferentes trayectos de los servicios de transporte

- **Ruta A - Trayecto 1: salida-carga**. Por carretera. Camión con un contenedor. Desde el núcleo urbano en Península o Canarias, exceptuando Gran Canaria, donde se imprimen los libros hasta el puerto más cercano. Metodología EN 16258.
- **Ruta A- Trayecto 2: carga-descarga**. Por mar. Buque de carga propio de la ruta que se cubre. Desde el puerto de carga hasta el de descarga en Las Palmas de Gran Canaria. Metodología Clean Cargo Workin Group.
- **Ruta A- Trayecto 3: descarga-llegada**. Por carretera. Camión de mercancías. Desde el Puerto de Las Palmas de Gran Canaria hasta el punto de entrega de la mercancía. Metodología EN 16258.
- **Ruta B- Trayecto1: salida-llegada**. Por carretera. Camión de mercancías. Desde el núcleo urbano en Gran Canaria donde se imprimen los libros hasta el punto de entrega de la mercancía. Metodología EN 16258.

In [65]:
salida_carga_distance = pd.read_csv('../data/processed/city_port_distance.csv')
salida_carga_distance.drop(columns=['Unnamed: 0', 'id', 'lat', 'lon'], inplace=True)
# Si el puerto más cercano es el de Las Palmas significa que la ciudad está en 
# Gran Canaria, por lo tanto se irá directamente al punto de despacho sin
# necesidad de ir al puerto (este trayecto será 0).
salida_carga_distance.loc[salida_carga_distance['port_name']=='LAS PALMAS', 'shortest_distance_to_port_km'] = 0 

carga_descarga_distance = pd.read_csv('../data/processed/port_portlp_distance.csv')
carga_descarga_distance.drop(columns='Unnamed: 0', inplace=True)

descarga_llegada_distance = pd.read_csv('../data/processed/puertodescarga_llegada_distance.csv')
descarga_llegada_distance.drop(columns='Unnamed: 0', inplace=True)

salida_llegada_distance = pd.read_csv('../data/processed/cityGC_destination_distance.csv')
salida_llegada_distance.drop(columns='Unnamed: 0', inplace=True)

In [66]:
# Dataframes
salida_carga = salida_carga_distance.copy()

carga_descarga = carga_descarga_distance.copy()

descarga_llegada = descarga_llegada_distance.copy()

salida_llegada = salida_llegada_distance.copy()

## Etapa 2: cálculo del consumo de energía y de emisiones de GEI de cada trayecto y servicio de transporte

### Subetapa 2.1: establecimiento del sistema de operación de un vehículo (VOS) relativo a cada trayecto

#### 1. Ruta A - Trayecto: salida-carga

1.	Vehículo. Un camión de 12-24 toneladas transporta un contenedor (1 TEU ) (Tabla 10, Schmied y Knörr, 2012) con mercancías de tipo medio .
2.	Combustible. El motor es de diésel y no se utilizan biocombustibles.
3.	Masa y factor de carga . El contenedor pesa 1,95 toneladas y lleva una carga de 10,5 toneladas, por lo que su peso total son 12,45 toneladas (Tabla 9, Schmied y Knörr, 2012). Este dato ya considera el factor de carga para mercancías de tipo medio.
4.	Consumo energético específico y viajes en vacío. El consumo energético específico para este camión transportando mercancías de tipo medio y desplazándose por terreno mayoritariamente plano es 0,24 litros/TEU-km (Tabla 12, Schmied y Knörr, 2012). Este consumo ya incluye un 10% de viajes en vacío  adicionales (Tabla 10, Schmied y Knörr, 2012). 
5.	Terreno. Se asume que el camión viaja mayoritariamente por terreno plano: autopistas o autovías. No se aplica factor de corrección por el uso de carreteras urbanas.

#### 2. Ruta A - Trayecto: carga-descarga

1.	Vehículo. Se desconoce el tipo de barco que realiza el transporte, por lo que un factor de emisión asociado al tipo de barco no es lo más adecuado. En lugar de esto, conocemos los puntos de salida y de llegada, así como la ruta. Así, un factor de emisión agregado para la ruta resulta más apropiado y esto es lo que provee Clean Cargo Working Group (Clean Cargo, 2021).
2.	Ruta (Clean Cargo, 2021)
a.	Trayecto desde Península: Europa (norte y mediterráneo) desde-hacia África.
b.	Trayecto desde Canarias (excepto Gran Canaria): Otra
3.	Factor de utilización. 70%.
4.	Distancia del trayecto. Según Clean Cargo Working Group resalta que las distancias interportuarias subestiman la distancia que realmente navegan los buques. Estudios realizados hasta la fecha indican que aplicando un factor de ajuste de un 15% mejora la precisión del cálculo de emisiones de GEI. En este estudio el factor se aplica a los trayectos desde península, pero no entre islas.
5.	Número de contenedores transportados por el buque. Los buques que cubren rutas intracontinentales europeas de media transportan 500-2000 TEU (Tabla 37, EcoTransIT World, 2023).
6.	Frigorífico. Sin contenedores frigoríficos.
7.	Masa. La metodología de CCWG considera que los contenedores tienen una masa neta media de 10 toneladas (CCWG, 2015). Por lo tanto, si asumimos que el contenedor tiene una masa de 1,95 t (Tabla 9, Schmied y Knörr, 2012)., la masa total sería de 11,95 t.

#### 3. Ruta A - Trayecto: descarga-llegada

1.	Vehículo. Un camión con un peso bruto menor a 7,5 t transporta mercancías medias.
2.	Combustible. El motor es de diésel y no se utilizan biocombustibles.
3.	Masa y factor de carga. El camión puede transportar como máximo 3,5 t, pero como el factor de carga es un 60%, la masa de la carga es 2.1 t.
4.	Consumo energético específico y viajes en vacío. El consumo energético específico para este camión transportando mercancías de tipo medio y desplazándose por terreno mayoritariamente plano es 0,077 litro/t-km (Tabla 12, Schmied y Knörr, 2012). Este consumo ya incluye un 20% de viajes en vacío adicionales (Tabla 10, Schmied y Knörr, 2012).
5.	Terreno. Se asume que el camión viaja mayoritariamente por terreno plano. Se aplica un factor de corrección de 0,9 por el uso de carreteras urbanas (Tabla 11, Schmied y Knörr, 2012).

#### 4. Ruta B - Trayecto salida-llegada

El camión que realiza este servicio es equivalente al del Trayecto 3: descarga-llegada.

### Subetapa 2.2: cuantificación del consumo total de combustible para cada VOS

Este cálculo se realiza únicamente para los trayectos por carretera, donde se utiliza la metodología EN 162158. 

#### 1. Ruta A - Trayecto salida-carga

In [67]:
# Definir funcion para el cálculo del consumo de diesel en base a la distancia.
# Se transporta un TEU, por esto no se pone actual cargo weight.
def salida_carga_diesel(distancia_trayecto):
    specific_energy_consumption_lTEUkm = 0.24
    diesel_l = distancia_trayecto * specific_energy_consumption_lTEUkm
    return diesel_l

In [68]:
salida_carga['consumo_diesel_TEU_l'] = salida_carga.apply(lambda row:salida_carga_diesel(row['shortest_distance_to_port_km']), axis=1)

#### 2. Ruta A - Trayecto carga-descarga

Este cálculo no es necesario para los trayectos por mar, donde se sigue la metodología de Clean Cargo Working Group.

#### 3. Ruta A - Trayecto descarga-llegada

In [69]:
# Definir funcion para el cálculo del consumo de diesel en base a la distancia.
def descarga_llegada_diesel(distancia_trayecto):
    specific_energy_consumption_ltkm = 0.077
    actual_cargo_weight_t = 2.1
    factor_correccion_carreteras_urbanas = 0.9
    diesel_l = (distancia_trayecto * actual_cargo_weight_t * specific_energy_consumption_ltkm) * factor_correccion_carreteras_urbanas
    return diesel_l

In [70]:
descarga_llegada['consumo_diesel_total_l'] = descarga_llegada.apply(lambda row:descarga_llegada_diesel(row['descarga_llegada_distancia_km']), axis=1)

#### 4. Ruta B - Trayecto salida-llegada

In [71]:
salida_llegada['consumo_diesel_total_l'] = salida_llegada.apply(lambda row:descarga_llegada_diesel(row['salida_llegada_distancia_km']), axis=1)

### Subetapa 2.3: cálculo del consumo total de energía y de emisiones de GEI para cada VOS

Este valor se obtiene de dos maneras diferentes, dependiendo de si se sigue la metodología EN 16258 (trayectos por tierra) o la de CCWG (trayectos por mar).
1.	Trayectos por tierra (EN 16258). En base al consumo de combustible calculado en la subetapa anterior, se calculan las emisiones de carbono (kgCO2e). Para esto se multiplica el consumo de diésel por los factores de emisión de fuente a ruedas (WTW) y de depósito a ruedas (TTW) (Tabla A.1, AENOR, 2013). Por otro lado, también se podría calcular el consumo de energía de total (MJ) multiplicando por los factores energéticos (WTW y TTW) (Tabla A.1, AENOR, 2013).
2.	Trayectos por mar (CCWG). Las emisiones de CO2.se calcula aplicando la fórmula general de CCWG y los factores emisión actualizados (CCWG, 2015; Clean Cargo, 2021).


#### 1. Trayecto salida-carga

##### Energía total

In [72]:
def carretera_consumo_energetico_TTW(consumo_diesel):
    factor_energetico_TTW_et_MJl = 35.9
    carretera_TTW_MJ = consumo_diesel * factor_energetico_TTW_et_MJl
    return carretera_TTW_MJ

In [73]:
def carretera_consumo_energetico_WTW(consumo_diesel):
    factor_energetico_WTW_ew_MJl = 42.7
    carretera_WTW_MJ = consumo_diesel * factor_energetico_WTW_ew_MJl
    return carretera_WTW_MJ

In [74]:
def carretera_consumo_energetico_WTT(consumo_diesel):
    wtw = carretera_consumo_energetico_WTW(consumo_diesel)
    ttw = carretera_consumo_energetico_TTW(consumo_diesel)
    wtt = wtw - ttw
    return wtt

##### Emisiones de carbono

In [75]:
# Datos (AENOR2013)
def carretera_emisiones_carbono_TTW(consumo_diesel):
    factor_emision_TTW_gt_kgCO2el = 2.67
    carretera_TTW_kgCO2e = consumo_diesel * factor_emision_TTW_gt_kgCO2el
    return carretera_TTW_kgCO2e

In [76]:
def carretera_emisiones_carbono_WTW(consumo_diesel):
    factor_emision_WTW_gw_kgCO2el = 3.24
    carretera_WTW_kgCO2e = consumo_diesel * factor_emision_WTW_gw_kgCO2el
    return carretera_WTW_kgCO2e

In [77]:
def carretera_emisiones_carbono_WTT(consumo_diesel):
    wtw = carretera_emisiones_carbono_WTW(consumo_diesel)
    ttw = carretera_emisiones_carbono_TTW(consumo_diesel)
    wtt = wtw - ttw
    return wtt

In [78]:
salida_carga['emisionesC_TTW_TEU_kgCO2e'] = salida_carga.apply(lambda row:carretera_emisiones_carbono_TTW(row['consumo_diesel_TEU_l']), axis=1)

salida_carga['emisionesC_WTT_TEU_kgCO2e'] = salida_carga.apply(lambda row:carretera_emisiones_carbono_WTT(row['consumo_diesel_TEU_l']), axis=1)

salida_carga['emisionesC_WTW_TEU_kgCO2e'] = salida_carga.apply(lambda row:carretera_emisiones_carbono_WTW(row['consumo_diesel_TEU_l']), axis=1)

#### 2. Trayecto carga-descarga

##### 1. Península

In [79]:
def mar_peninsula_emisiones_carbono_WTW(distancia_trayecto):
    emission_factor_WTW_gCO2TEUkm = 100.2
    detour = distancia_trayecto * 0.15
    teus_shipment = 1 
    mar_WTW_gCO2e = emission_factor_WTW_gCO2TEUkm * (distancia_trayecto + detour) * teus_shipment
    mar_WTW_kgCO2e = mar_WTW_gCO2e/1000
    return mar_WTW_kgCO2e

In [80]:
def mar_peninsula_emisiones_carbono_TTW(distancia_trayecto):
    emission_factor_TTW_gCO2TEUkm = 59.7
    detour = distancia_trayecto * 0.15
    teus_shipment = 1
    mar_TTW_gCO2e = (emission_factor_TTW_gCO2TEUkm * (distancia_trayecto + detour) * teus_shipment) / 0.7
    mar_TTW_kgCO2e = mar_TTW_gCO2e/1000
    return mar_TTW_kgCO2e

In [81]:
def mar_peninsula_emisiones_carbono_WTT(distancia_trayecto):
    wtw = mar_peninsula_emisiones_carbono_WTW(distancia_trayecto)
    ttw = mar_peninsula_emisiones_carbono_TTW(distancia_trayecto)
    wtt = wtw - ttw
    return wtt

##### 2. Canarias

In [82]:
def mar_canarias_emisiones_carbono_WTW(distancia_trayecto):
    emission_factor_WTW_gCO2TEUkm = 110.9
    detour = distancia_trayecto * 0.15
    teus_shipment = 1 
    mar_WTW_gCO2e = emission_factor_WTW_gCO2TEUkm * (distancia_trayecto + detour) * teus_shipment
    mar_WTW_kgCO2e = mar_WTW_gCO2e/1000
    return mar_WTW_kgCO2e

In [83]:
def mar_canarias_emisiones_carbono_TTW(distancia_trayecto):
    emission_factor_TTW_gCO2TEUkm = 66.8
    detour = distancia_trayecto * 0.15
    teus_shipment = 1
    mar_TTW_gCO2e = (emission_factor_TTW_gCO2TEUkm * (distancia_trayecto + detour) * teus_shipment) / 0.7
    mar_TTW_kgCO2e = mar_TTW_gCO2e/1000
    return mar_TTW_kgCO2e

In [84]:
def mar_canarias_emisiones_carbono_WTT(distancia_trayecto):
    wtw = mar_canarias_emisiones_carbono_WTW(distancia_trayecto)
    ttw = mar_canarias_emisiones_carbono_TTW(distancia_trayecto)
    wtt = wtw - ttw
    return wtt

Aplicamos las funciones.

In [85]:
# Diferentes suposiciones dependiendo del puerto de salida
canary_ports = ['TENERIFE', 'ARRECIFE', 'LA PALMA', 'LA ESTACA', 'PUERTO DEL ROSARIO', 'LA GOMERA', 'LAS PALMAS']

carga_descarga['flag'] = carga_descarga['port_name'].isin(canary_ports) * 1

In [86]:
carga_descarga['emisionesC_TTW_TEU_kgCO2e'] = carga_descarga.apply(lambda row:mar_peninsula_emisiones_carbono_TTW(row['distanceLP_km']) if row.flag==0
                                                                   else mar_canarias_emisiones_carbono_TTW(row['distanceLP_km']), axis=1)

carga_descarga['emisionesC_WTT_TEU_kgCO2e'] = carga_descarga.apply(lambda row:mar_peninsula_emisiones_carbono_WTT(row['distanceLP_km']) if row.flag==0
                                                                   else mar_canarias_emisiones_carbono_WTT(row['distanceLP_km']), axis=1)

carga_descarga['emisionesC_WTW_TEU_kgCO2e'] = carga_descarga.apply(lambda row:mar_peninsula_emisiones_carbono_WTW(row['distanceLP_km']) if row.flag==0
                                                                   else mar_canarias_emisiones_carbono_WTW(row['distanceLP_km']), axis=1)

#### 3. Trayecto descarga-llegada

##### Emisiones de carbono

Aprovechamos las funciones definidas para el Trayecto 1: salida_carga.

In [87]:
descarga_llegada['emisionesC_TTW_total_kgCO2e'] = descarga_llegada.apply(lambda row:carretera_emisiones_carbono_TTW(row['consumo_diesel_total_l']), axis=1)

descarga_llegada['emisionesC_WTT_total_kgCO2e'] = descarga_llegada.apply(lambda row:carretera_emisiones_carbono_WTT(row['consumo_diesel_total_l']), axis=1)

descarga_llegada['emisionesC_WTW_total_kgCO2e'] = descarga_llegada.apply(lambda row:carretera_emisiones_carbono_WTW(row['consumo_diesel_total_l']), axis=1)

#### 4. Trayecto salida-llegada

Aprovechamos las funciones definidas para el Trayecto 1: salida_carga.

In [88]:
salida_llegada['emisionesC_TTW_total_kgCO2e'] = salida_llegada.apply(lambda row:carretera_emisiones_carbono_TTW(row['consumo_diesel_total_l']), axis=1)

salida_llegada['emisionesC_WTT_total_kgCO2e'] = salida_llegada.apply(lambda row:carretera_emisiones_carbono_WTT(row['consumo_diesel_total_l']), axis=1)

salida_llegada['emisionesC_WTW_total_kgCO2e'] = salida_llegada.apply(lambda row:carretera_emisiones_carbono_WTW(row['consumo_diesel_total_l']), axis=1)

### Subetapa 2.4: asignación a cada trayecto de una porción de cada uno de los resultados de la subetapa 2.3

En esta subetapa se asigna una parte de las emisiones y de la energía ya calculadas a la carga de libros. Esto se realiza en base a su participación relativa en la actividad de transporte realizada dentro del VOS. Para esto se calcula un factor de proporción de carga (fc) que luego se multiplicará por las emisiones o energía total. La carga de libros siempre será de 500 ejemplares e incluye la masa de un europalé de 25 kg (Tabla 1).

f_(c =)  m_l/m_T 

Donde:
- fc = Factor de proporción de carga (adimensional).
- ml = Masa de las 500 unidades de libros (kg). Incluye la masa de un europalé de 25 kg. Tabla 1.
- mT = Masa total del cargamento (kg). Valor según trayecto:
- Ruta A – Trayecto 1: salida-carga = 12,45 t.
- Ruta A – Trayecto 2: carga-descarga = 11,95 t. 
- Ruta A – Trayecto 3: descarga-llegada = 2,1 t.
- Ruta B – Trayecto 1: salida-llegada = 2,1. Se usa la misma función que para el Trayecto 3: descarga-llegada.

A continuación, se calcula la proporción de las emisiones de GEI o de la energía total correspondiente al transporte de libros.

P= f_c×C

Donde:
- P = Proporción de las emisiones de GEI (kg CO2e) o de la energía total (MJ) correspondiente al transporte de la carga de libros. Este paso se realiza- por separado de fuente a ruedas (WTW), de depósito a ruedas (TTW) y de fuente a depósito (WTT).
- fc = Factor de proporción de carga (adimensional).
- C = Emisiones de gases (kg CO2e) o energía total (MJ) calculados en la subetapa 2.3 (apartado 4.2.3 más atrás).


#### 1. Trayecto salida-carga

##### Factor proporción libros

In [89]:
# Añadir peso de los libros
salida_carga = pd.merge(salida_carga, libros[['libro_titulo', 'peso_t_500libros']], how='cross')

In [90]:
def salida_carga_factor_proporcion_carga(peso_mercancia):
    actual_cargo_weight_t = 12.45
    factor = peso_mercancia / actual_cargo_weight_t
    return factor

##### Energía total

In [91]:
def salida_carga_consumo_energetico_proporcion_TTW(peso_mercancia, consumo_diesel):
    factor = salida_carga_factor_proporcion_carga(peso_mercancia)
    total = carretera_consumo_energetico_TTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [92]:
def salida_carga_consumo_energetico_proporcion_WTW(peso_mercancia, consumo_diesel):
    factor = salida_carga_factor_proporcion_carga(peso_mercancia)
    total = carretera_consumo_energetico_WTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [93]:
def salida_carga_consumo_energetico_proporcion_WTT(peso_mercancia, consumo_diesel):
    factor = salida_carga_factor_proporcion_carga(peso_mercancia)
    total = carretera_consumo_energetico_WTT(consumo_diesel)
    proporcion = factor * total
    return proporcion

##### Emisiones carbono

In [94]:
def salida_carga_emisiones_carbono_proporcion_TTW(peso_mercancia, consumo_diesel):
    factor = salida_carga_factor_proporcion_carga(peso_mercancia)
    total = carretera_emisiones_carbono_TTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [95]:
def salida_carga_emisiones_carbono_proporcion_WTW(peso_mercancia, consumo_diesel):
    factor = salida_carga_factor_proporcion_carga(peso_mercancia)
    total = carretera_emisiones_carbono_WTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [96]:
def salida_carga_emisiones_carbono_proporcion_WTT(peso_mercancia, consumo_diesel):
    factor = salida_carga_factor_proporcion_carga(peso_mercancia)
    total = carretera_emisiones_carbono_WTT(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [97]:
# El contendor pesa en total 12.45 tonnes (actual_cargo_weight_t)
salida_carga['emisionesC_TTW_libros_kgCO2e'] = salida_carga.apply(lambda row:salida_carga_emisiones_carbono_proporcion_TTW(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_TEU_l']), axis=1)

salida_carga['emisionesC_WTT_libros_kgCO2e'] = salida_carga.apply(lambda row:salida_carga_emisiones_carbono_proporcion_WTT(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_TEU_l']), axis=1)

salida_carga['emisionesC_WTW_libros_kgCO2e'] = salida_carga.apply(lambda row:salida_carga_emisiones_carbono_proporcion_WTW(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_TEU_l']), axis=1)

#### 2. Trayecto carga-descarga

##### Factor proporción libros

In [98]:
# Añadir peso de los libros
carga_descarga = pd.merge(carga_descarga, libros[['libro_titulo', 'peso_t_500libros']], how='cross')

In [99]:
# La metodología de Clean Cargo asume que 1 TEU = 10 toneladas
def mar_factor_proporcion_carga(peso_mercancia):
    actual_cargo_weight_t = 11.95
    factor = peso_mercancia / actual_cargo_weight_t
    return factor

##### Emisiones de carbono

###### 1. Península

In [100]:
def mar_peninsula_emisiones_carbono_proporcion_TTW(peso_mercancia, distancia_trayecto):
    factor = mar_factor_proporcion_carga(peso_mercancia)
    total = mar_peninsula_emisiones_carbono_TTW(distancia_trayecto)
    proporcion = factor * total
    return proporcion

In [101]:
def mar_peninsula_emisiones_carbono_proporcion_WTW(peso_mercancia, distancia_trayecto):
    factor = mar_factor_proporcion_carga(peso_mercancia)
    total = mar_peninsula_emisiones_carbono_WTW(distancia_trayecto)
    proporcion = factor * total
    return proporcion

In [102]:
def mar_peninsula_emisiones_carbono_proporcion_WTT(peso_mercancia, distancia_trayecto):
    factor = mar_factor_proporcion_carga(peso_mercancia)
    total = mar_peninsula_emisiones_carbono_WTT(distancia_trayecto)
    proporcion = factor * total
    return proporcion

###### 2. Canarias

In [103]:
def mar_canarias_emisiones_carbono_proporcion_TTW(peso_mercancia, distancia_trayecto):
    factor = mar_factor_proporcion_carga(peso_mercancia)
    total = mar_canarias_emisiones_carbono_TTW(distancia_trayecto)
    proporcion = factor * total
    return proporcion

In [104]:
def mar_canarias_emisiones_carbono_proporcion_WTW(peso_mercancia, distancia_trayecto):
    factor = mar_factor_proporcion_carga(peso_mercancia)
    total = mar_canarias_emisiones_carbono_WTW(distancia_trayecto)
    proporcion = factor * total
    return proporcion

In [105]:
def mar_canarias_emisiones_carbono_proporcion_WTT(peso_mercancia, distancia_trayecto):
    factor = mar_factor_proporcion_carga(peso_mercancia)
    total = mar_canarias_emisiones_carbono_WTT(distancia_trayecto)
    proporcion = factor * total
    return proporcion

In [106]:
carga_descarga['emisionesC_TTW_libros_kgCO2e'] = carga_descarga.apply(lambda row:mar_peninsula_emisiones_carbono_proporcion_TTW(peso_mercancia=row['peso_t_500libros'], distancia_trayecto=row['distanceLP_km']) if row.flag==0
                                                                   else mar_canarias_emisiones_carbono_proporcion_TTW(peso_mercancia=row['peso_t_500libros'], distancia_trayecto=row['distanceLP_km']), axis=1)

carga_descarga['emisionesC_WTT_libros_kgCO2e'] = carga_descarga.apply(lambda row:mar_peninsula_emisiones_carbono_proporcion_WTT(peso_mercancia=row['peso_t_500libros'], distancia_trayecto=row['distanceLP_km']) if row.flag==0
                                                                   else mar_canarias_emisiones_carbono_proporcion_WTT(peso_mercancia=row['peso_t_500libros'], distancia_trayecto=row['distanceLP_km']), axis=1)

carga_descarga['emisionesC_WTW_libros_kgCO2e'] = carga_descarga.apply(lambda row:mar_peninsula_emisiones_carbono_proporcion_WTW(peso_mercancia=row['peso_t_500libros'], distancia_trayecto=row['distanceLP_km']) if row.flag==0
                                                                   else mar_canarias_emisiones_carbono_proporcion_WTW(peso_mercancia=row['peso_t_500libros'], distancia_trayecto=row['distanceLP_km']), axis=1)

#### 3. Trayecto descarga-llegada

In [107]:
# Añadir peso de los libros
descarga_llegada = pd.merge(descarga_llegada, libros[['libro_titulo', 'peso_t_500libros']], how='cross')

##### Factor proporción libros

In [108]:
def carreteraGC_factor_proporcion_carga(peso_mercancia):
    actual_cargo_weight_t = 2.1
    factor = peso_mercancia / actual_cargo_weight_t
    return factor

##### Energía total

In [109]:
def carreteraGC_consumo_energetico_proporcion_TTW(peso_mercancia, consumo_diesel):
    factor = carreteraGC_factor_proporcion_carga(peso_mercancia)
    total = carretera_consumo_energetico_TTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [110]:
def carreteraGC_consumo_energetico_proporcion_WTW(peso_mercancia, consumo_diesel):
    factor = carreteraGC_factor_proporcion_carga(peso_mercancia)
    total = carretera_consumo_energetico_WTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [111]:
def carreteraGC_consumo_energetico_proporcion_WTT(peso_mercancia, consumo_diesel):
    factor = carreteraGC_factor_proporcion_carga(peso_mercancia)
    total = carretera_consumo_energetico_WTT(consumo_diesel)
    proporcion = factor * total
    return proporcion

##### Emisiones carbono

In [112]:
def carreteraGC_emisiones_carbono_proporcion_TTW(peso_mercancia, consumo_diesel):
    factor = carreteraGC_factor_proporcion_carga(peso_mercancia)
    total = carretera_emisiones_carbono_TTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [113]:
def carreteraGC_emisiones_carbono_proporcion_WTW(peso_mercancia, consumo_diesel):
    factor = carreteraGC_factor_proporcion_carga(peso_mercancia)
    total = carretera_emisiones_carbono_WTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [114]:
def carreteraGC_emisiones_carbono_proporcion_WTT(peso_mercancia, consumo_diesel):
    factor = carreteraGC_factor_proporcion_carga(peso_mercancia)
    total = carretera_emisiones_carbono_WTT(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [115]:
descarga_llegada['emisionesC_TTW_libros_kgCO2e'] = descarga_llegada.apply(lambda row:carreteraGC_emisiones_carbono_proporcion_TTW(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)

descarga_llegada['emisionesC_WTT_libros_kgCO2e'] = descarga_llegada.apply(lambda row:carreteraGC_emisiones_carbono_proporcion_WTT(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)

descarga_llegada['emisionesC_WTW_libros_kgCO2e'] = descarga_llegada.apply(lambda row:carreteraGC_emisiones_carbono_proporcion_WTW(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)

#### 4. Trayecto salida-llegada

In [116]:
# Añadir peso de los libros
salida_llegada = pd.merge(salida_llegada, libros[['libro_titulo', 'peso_t_500libros']], how='cross')

Se usan las funciones definidas en el Trayecto descarga-llegada.

In [117]:
salida_llegada['emisionesC_TTW_libros_kgCO2e'] = salida_llegada.apply(lambda row:carreteraGC_emisiones_carbono_proporcion_TTW(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)

salida_llegada['emisionesC_WTT_libros_kgCO2e'] = salida_llegada.apply(lambda row:carreteraGC_emisiones_carbono_proporcion_WTT(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)

salida_llegada['emisionesC_WTW_libros_kgCO2e'] = salida_llegada.apply(lambda row:carreteraGC_emisiones_carbono_proporcion_WTW(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)

In [118]:
salida_llegada_GC = salida_llegada.copy()

## Etapa 3: suma de los resultados de cada trayecto

### Puntos de inicio fuera de Gran Canaria

In [119]:
salida_carga.drop(columns=['consumo_diesel_TEU_l',
'emisionesC_TTW_TEU_kgCO2e', 'emisionesC_WTT_TEU_kgCO2e', 'emisionesC_WTW_TEU_kgCO2e'], inplace=True)

# Renombrar columnas para unir
salida_carga.rename(columns={'port_name': 'port_name_salida',
'emisionesC_TTW_libros_kgCO2e': 'emisionesC_TTW_libros_kgCO2e_SC',
'emisionesC_WTT_libros_kgCO2e': 'emisionesC_WTT_libros_kgCO2e_SC', 
'emisionesC_WTW_libros_kgCO2e': 'emisionesC_WTW_libros_kgCO2e_SC'}, inplace=True)

In [120]:
carga_descarga.drop(columns=['distanceLP_nm', 'flag',
'emisionesC_TTW_TEU_kgCO2e', 'emisionesC_WTT_TEU_kgCO2e', 'emisionesC_WTW_TEU_kgCO2e'], inplace=True)

# Renombrar columnas para unir
carga_descarga.rename(columns={'port_name': 'port_name_salida',
'emisionesC_TTW_libros_kgCO2e': 'emisionesC_TTW_libros_kgCO2e_CD',
'emisionesC_WTT_libros_kgCO2e': 'emisionesC_WTT_libros_kgCO2e_CD', 
'emisionesC_WTW_libros_kgCO2e': 'emisionesC_WTW_libros_kgCO2e_CD'}, inplace=True)

In [121]:
descarga_llegada.drop(columns=['port_latlon', 'consumo_diesel_total_l', 
'emisionesC_TTW_total_kgCO2e', 'emisionesC_WTT_total_kgCO2e', 'emisionesC_WTW_total_kgCO2e'], inplace=True)

# Renombrar columnas para unir
descarga_llegada.rename(columns={'port_name': 'port_name_llegada',
'emisionesC_TTW_libros_kgCO2e': 'emisionesC_TTW_libros_kgCO2e_DL',
'emisionesC_WTT_libros_kgCO2e': 'emisionesC_WTT_libros_kgCO2e_DL', 
'emisionesC_WTW_libros_kgCO2e': 'emisionesC_WTW_libros_kgCO2e_DL'}, inplace=True)

In [122]:
# Unir trayectos
salida_descarga_noGC = pd.merge(salida_carga, carga_descarga, on=['port_name_salida', 'libro_titulo', 'peso_t_500libros'])

salida_llegada_noGC = pd.merge(salida_descarga_noGC, descarga_llegada, on=['libro_titulo', 'peso_t_500libros'])

# Elimino los puntos de salida que son desde GC
salida_llegada_noGC = salida_llegada_noGC[salida_llegada_noGC['port_name_salida'] !='LAS PALMAS']

# Suma ruta distancia total
salida_llegada_noGC["distancia_ruta_km"] = salida_llegada_noGC['shortest_distance_to_port_km'] + salida_llegada_noGC['distanceLP_km']
+ salida_llegada_noGC['descarga_llegada_distancia_km']
# Suma ruta emisiones
salida_llegada_noGC['emisionesC_TTW_libros_kgCO2e_ruta'] = salida_llegada_noGC['emisionesC_TTW_libros_kgCO2e_SC'] + salida_llegada_noGC['emisionesC_TTW_libros_kgCO2e_CD'] + salida_llegada_noGC['emisionesC_TTW_libros_kgCO2e_DL']
salida_llegada_noGC['emisionesC_WTT_libros_kgCO2e_ruta'] = salida_llegada_noGC['emisionesC_WTT_libros_kgCO2e_SC'] + salida_llegada_noGC['emisionesC_WTT_libros_kgCO2e_CD'] + salida_llegada_noGC['emisionesC_WTT_libros_kgCO2e_DL']
salida_llegada_noGC['emisionesC_WTW_libros_kgCO2e_ruta'] = salida_llegada_noGC['emisionesC_WTW_libros_kgCO2e_SC'] + salida_llegada_noGC['emisionesC_WTW_libros_kgCO2e_CD'] + salida_llegada_noGC['emisionesC_WTW_libros_kgCO2e_DL']

### Añadir puntos de inicio en Gran Canaria

In [123]:
salida_llegada_GC.drop(columns=['consumo_diesel_total_l',
'emisionesC_TTW_total_kgCO2e', 'emisionesC_WTT_total_kgCO2e', 'emisionesC_WTW_total_kgCO2e'], inplace=True)

# Renombrar columnas para unir
salida_llegada_GC.rename(columns={'salida_llegada_distancia_km': 'distancia_ruta_km',
'emisionesC_TTW_libros_kgCO2e': 'emisionesC_TTW_libros_kgCO2e_ruta',
'emisionesC_WTT_libros_kgCO2e': 'emisionesC_WTT_libros_kgCO2e_ruta', 
'emisionesC_WTW_libros_kgCO2e': 'emisionesC_WTW_libros_kgCO2e_ruta'}, inplace=True)

Ordenar y concatenar resultados

In [124]:
salida_llegada_noGC = salida_llegada_noGC[['name', 'population', 'latlon', 'port_name_salida', 'port_latlon', 
'punto_llegada_latlon', 'libro_titulo', 'peso_t_500libros', 'distancia_ruta_km', 
'emisionesC_TTW_libros_kgCO2e_ruta', 'emisionesC_WTT_libros_kgCO2e_ruta', 'emisionesC_WTW_libros_kgCO2e_ruta']]

salida_llegada_GC = salida_llegada_GC[['name', 'population', 'latlon', 'punto_llegada_latlon', 'libro_titulo', 'peso_t_500libros',
'distancia_ruta_km', 'emisionesC_TTW_libros_kgCO2e_ruta', 'emisionesC_WTT_libros_kgCO2e_ruta', 'emisionesC_WTW_libros_kgCO2e_ruta']]

resultados_huella = pd.concat([salida_llegada_GC, salida_llegada_noGC]).reset_index(drop=True)

resultados_huella.to_csv('../data/final/resultados_huella.csv')